In [22]:
# necessary imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.applications import vgg16
from IPython.display import Image, display
# define constants and paths
c_img_path = 'random.jpg'
s_img_path = 'mona_lisa.jpg'

style_weight = 1e-5
content_weight = 3.5e-8

#Here I am getting the dimensions of the content image
w,h=keras.utils.load_img(c_img_path).size

#Here I am scaling the image according to my preference keeping the aspect ratio same.
n_rows = 800
n_cols = int((w * n_rows)/h)

#VGG 16 preprocess function

def preprocess_img(img_path):
    img=keras.utils.load_img(img_path,target_size=(n_rows,n_cols))
    #img=keras.utils.img_to_array(img)
    img=np.array([img])
    img=vgg16.preprocess_input(img)
    return img

img=preprocess_img('mona_lisa.jpg')

def deprocess_img(x):
# input - BGR image with means subtracted
    x = x.reshape((n_rows, n_cols, 3))
    # mean BGR values from imagenet - subtracted in vgg preprocess
    mean_imagenet = [103.939, 116.779, 123.68]
    # re-add means
    for i, val in enumerate(mean_imagenet):
        x[:, :, i] += val
    # restore RGB from BGR using slicing array
    x = x[:, :, ::-1] # channels put in reverse order
    
    return x

def f_extractor(layer_name,model):
    f=keras.Model(inputs=model.inputs,outputs=model.get_layer(name=layer_name).output)
    return f

def content_loss(g_f,c_f):
    return tf.reduce_sum(tf.square(g_f-c_f))


def gram_matrix(features):
    #Here I made channels the first dimension
    features = tf.transpose(features, (2, 0, 1))

    #Make the above matrix 2d. keep the channel dimension
    features = tf.reshape(features, (tf.shape(features)[0], -1))

    # Making Gram Matrix G= A* A_T
    gram_matrix = tf.matmul(features, tf.transpose(features))

    # Normalizing loss
    N = features.shape[0]
    M = features.shape[1]
    denominator = 2 * N  * M
    # return the scaled matrix
    return gram_matrix / tf.cast(denominator, tf.float32)

def style_loss(g_features, s_features):
# calculate style loss
    Gen = gram_matrix(g_features[0]) 
    print(g_features.shape[0])
    print(g_features[0].shape[0])
    Style = gram_matrix(s_features[0])
    return tf.reduce_sum(tf.square(Style-Gen))


In [23]:

def total_loss(g_img, c_img, s_img):

    # define layers for content and style
    c_layer_name = 'block5_conv2'
    s_layer_names = [
        "block1_conv1",
        "block2_conv1",
        "block3_conv1",
        "block4_conv1",
        "block5_conv1",]

    # extract feature maps for content
    feature_extractor = f_extractor(c_layer_name, model)
    c_features = feature_extractor(c_img)
    g_c_features = feature_extractor(g_img)

    # extract feature maps for style
    s_features = []
    g_s_features = []
    for s_layer in s_layer_names:
        feature_extractor = f_extractor(s_layer, model)
        s_features.append(feature_extractor(s_img))
        g_s_features.append(feature_extractor(g_img))

    # define loss variable
    loss = tf.zeros(shape=())

    # calculate content loss
    loss += content_weight * content_loss(g_c_features, c_features)

    # calculate style loss
    for i, s_feature in enumerate(s_features):
        s_loss = style_loss(g_s_features[i], s_feature)
        loss += style_weight * s_loss / len(s_features)

    return loss





def loss_gradient(g_img, c_img, s_img):
    with tf.GradientTape() as tape:
        loss = total_loss(g_img, c_img, s_img)
    grads = tape.gradient(loss, g_img)
    return loss, grads

# load pretrained VGG16 model
model = vgg16.VGG16(weights='imagenet', include_top=False)

g_img = tf.Variable(preprocess_img(c_img_path))
# open and preprocess images with content and style
c_img = preprocess_img(c_img_path)
s_img = preprocess_img(s_img_path)

optimizer = keras.optimizers.Adam(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=32.0, decay_steps=100,
        decay_rate=0.98))

iterations = 1000
for i in range(1, iterations+1):

    loss, grads = loss_gradient(g_img, c_img, s_img)
    print(i, loss)
    optimizer.apply_gradients([(grads, g_img)])
    if i % 100 == 0:
        new_img = deprocess_img(np.array(g_img[0]))
        fname = "output_%d.png" % i
        keras.utils.save_img(fname, new_img)

KeyboardInterrupt: 

In [1]:
import tensorflow as tf

# Check if TensorFlow is built with GPU support
gpu_available = tf.test.is_built_with_cuda()

# List all physical devices of type GPU
gpus = tf.config.list_physical_devices('GPU')

if gpu_available and gpus:
    print("TensorFlow is running on the GPU.")
    print("Available GPU(s):", gpus)
else:
    print("TensorFlow is not running on the GPU.")

TensorFlow is not running on the GPU.


In [6]:
style='0.00001'
style=float(style)
style

1e-05

In [7]:
import pandas as pd

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [1]:
import numpy
numpy.version.version

'1.21.6'